In [ ]:
#Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import numpy as np
import os
import time

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from pathlib import Path
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from openpyxl import load_workbook

In [ ]:
#Reading the excel-document into a df
df = pd.read_excel('dataset_name.xlsx')

#dropping unwanted columns from the df
df2=df.drop(columns = ["Column 1", "Column 2"])

In [ ]:
#Splitting df into two based on ID
df2["ID"] = df2["ID"].astype(str)  #Making sure its a string
df_train = df2[df2["ID"].isin(["ID 1", "ID 2", "ID 3"])]
df_unseen = df2[df2["ID"] == "ID 4"]


In [ ]:
#Preparing PCA input by defining the columns to include in the transformation
columns = ['Column 1', 'Column 2', 'Column 3', 'Column 4', 'Column 5']

#Extracting and copying the selected columns from the training dataframe
data_train = df_train[columns].copy()

#Encoding 'ID' 
data_train['ID'] = data_train['ID'].astype('Category_column').cat.codes

#Initializing a standard scaler to normalize the data
scaler_pca = StandardScaler()

#Fitting the scaler on training data and apply the transformation
data_train_scaled = scaler_pca.fit_transform(data_train)

#Initializing PCA to reduce dimensionality to 4 PCs
pca = PCA(n_components=4)

#Fitting PCA on the scaled training data and transforming it
pca_train = pca.fit_transform(data_train_scaled)

#Adding the first 4 principal components (PC1 to PC4) as new columns to df_train
for i in range(4):
    df_train.loc[:, f'PC{i+1}'] = pca_train[:, i]


#Applying the same PCA transformation to the unseen dataset 

#Extracting and copying the same columns from the unseen dataset
data_unseen = df_unseen[columns].copy()

#Encode 'ID' 
data_unseen['ID'] = data_unseen['ID'].astype('Category_column').cat.codes

#Applying the same scaling 
data_unseen_scaled = scaler_pca.transform(data_unseen)

#Applying the same PCA transformation
pca_unseen = pca.transform(data_unseen_scaled)

#Adding PC1 to PC4 as new columns to df_unseen
for i in range(4):
    df_unseen.loc[:, f'PC{i+1}'] = pca_unseen[:, i]

In [ ]:
#Preparing inputs and target values
X = df_train[['PC1', 'PC2', 'PC3', 'PC4']]
y = df_train['Category_column']

X_unseen = df_unseen[['PC1', 'PC2', 'PC3', 'PC4']]
y_unseen = df_unseen['Category_column']

In [ ]:
#Setting up K-Fold cross-validation with 10 folds
k_fold = KFold(n_splits=10, random_state=66, shuffle=True)

#Defining a pipeline that first standardizes the input features, then applies a Random Forest Regressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),                     
    ('rf', RandomForestRegressor(                    
        n_estimators=40,                               
        max_depth=7,                                
        random_state=40 ))])

#Creating empty lists to store metrics
mae_scores_train, mse_scores_train, r2_scores_train = [], [], []
mae_scores_test, mse_scores_test, r2_scores_test = [], [], []

#Looping through each fold in the K-Fold split
for train_index, test_index in k_fold.split(X, y):
    #Splitting the data into training and test sets for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #Skipping fold if test target values contain only one unique value 
    if len(np.unique(y_test)) == 1:
        print("Skipping this fold due to only one class in test set.")
        continue

    #Fitting the pipeline (scaler + regressor) to the training data
    pipeline.fit(X_train, y_train)

    #Making predictions on both training and test data
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    #Calculating regression metrics for the training set
    mae_train = mean_absolute_error(y_train, pred_train)  
    mse_train = mean_squared_error(y_train, pred_train)      
    r2_train = r2_score(y_train, pred_train)                

    #Calculating regression metrics for the test set
    mae_test = mean_absolute_error(y_test, pred_test)
    mse_test = mean_squared_error(y_test, pred_test)
    r2_test = r2_score(y_test, pred_test)

    #Saving all metrics into the lists
    mae_scores_train.append(mae_train)
    mse_scores_train.append(mse_train)
    r2_scores_train.append(r2_train)

    mae_scores_test.append(mae_test)
    mse_scores_test.append(mse_test)
    r2_scores_test.append(r2_test)

    #Printing the performance for the current fold
    print(f"Train -> MAE: {mae_train:.6f}, MSE: {mse_train:.6f}, R²: {r2_train:.6f}")
    print(f"Test  -> MAE: {mae_test:.6f}, MSE: {mse_test:.6f}, R²: {r2_test:.6f}\n")

#After all folds, calculating and printing the average performance metrics
print("\nAverage results from all folds:")
print(f"Train Mean MAE: {np.mean(mae_scores_train):.6f}, Test Mean MAE: {np.mean(mae_scores_test):.6f}")
print(f"Train Mean MSE: {np.mean(mse_scores_train):.6f}, Test Mean MSE: {np.mean(mse_scores_test):.6f}")
print(f"Train Mean R²: {np.mean(r2_scores_train):.6f}, Test Mean R²: {np.mean(r2_scores_test):.6f}")


In [ ]:
#Defining a pipeline that includes scaling and the Random Forest Regressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),                  
    ('rf', RandomForestRegressor(random_state=66))  
])

#Defining the hyperparameter grid 
parameters_grid = {
    'rf__n_estimators': [100,200, 300],          
    'rf__max_features': [3, 5, 7],           
    'rf__max_depth': [7, 10, 15],            
    'rf__min_samples_split': [3,5,10],        
    'rf__min_samples_leaf': [3,5,10],         
    'rf__criterion': ['poisson', 'squared_error', 'absolute_error'],          
}

#Setting up GridSearchCV 
CV_RF = GridSearchCV(
    estimator=pipeline,            
    param_grid=parameters_grid,     
    cv=k_fold,                       
    scoring='r2',                   
    n_jobs=-1                       )

#Fitting GridSearchCV to the training data
CV_RF.fit(X_train, y_train)

#Printingthe best parameter combination found 
print('Best parameters: ', CV_RF.best_params_)


In [ ]:
#Defining the scaler and scaling train and test data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)     
X_test_scaled = scaler.transform(X_test)          

#Getting best hyperparameters from GridSearchCV 
best_params = {key.replace("rf__", ""): value for key, value in CV_RF.best_params_.items()}

#Initializing a new RFR using the best parameters
RF_best = RandomForestRegressor(**best_params)

#Training the model on the scaled training data
RF_best.fit(X_train_scaled, y_train)

#Making predictions on the scaled training and test datasets
y_train_pred = RF_best.predict(X_train_scaled)
y_test_pred = RF_best.predict(X_test_scaled)

#Evaluating model performance on training data
mse_train = mean_squared_error(y_train, y_train_pred)  
r2_train = r2_score(y_train, y_train_pred)             

#Evaluating model performance on test data
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

#Printing performance metrics for both training and test sets
print(f"Train Mean Squared Error: {mse_train:.4f}")
print(f"Train R-squared: {r2_train:.4f}")
print(f"Test Mean Squared Error: {mse_test:.4f}")
print(f"Test R-squared: {r2_test:.4f}")


In [ ]:
#Scaling the unseen data
X_unseen_scaled = scaler.transform(X_unseen)

#Predicting target values for the unseen data 
y_unseen_pred = RF_best.predict(X_unseen_scaled)

# Calculating performance metrics for the unseen data
mse_unseen = mean_squared_error(y_unseen, y_unseen_pred)
r2_unseen = r2_score(y_unseen, y_unseen_pred)

#Printing performance results on unseen data
print(f'Unseen Mean Squared Error: {mse_unseen:.4f}')
print(f'Unseen R-squared: {r2_unseen:.4f}')


In [ ]:
#Defining a function to log model evaluation metrics and hyperparameters to an Excel-file of chosen name
def log_results(model_name, params, mse_train, r2_train, mse_test, r2_test, mse_unseen, r2_unseen, filename="Results.xlsx"):
    """ Logs model results in separate sheets within the same Excel file, ensuring appending works correctly. """

    #Creating DataFrame for this model run
    result = pd.DataFrame([{
        **params,
        "MSE_Train": mse_train,
        "R2_Train": r2_train,
        "MSE_Test": mse_test,
        "R2_Test": r2_test,
        "MSE_Unseen": mse_unseen,
        "R2_Unseen": r2_unseen
    }])

    #Introducing a short delay to avoid file conflicts if running in multiple notebooks
    time.sleep(1)

    #Checking if the file exists
    file_exists = os.path.exists(filename)

    if not file_exists:
        #If file doesn't exist, creating a new one
        with pd.ExcelWriter(filename, engine="openpyxl", mode="w") as writer:
            result.to_excel(writer, sheet_name=model_name, index=False)
    else:
        #If file exists, loading it properly before appending
        try:
            with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
                #Reading existing sheet
                try:
                    existing_df = pd.read_excel(filename, sheet_name=model_name, engine="openpyxl")
                    df_combined = pd.concat([existing_df, result], ignore_index=True)
                except (FileNotFoundError, ValueError):
                    df_combined = result  #If sheet does not exist, creating it

                #Saving results
                df_combined.to_excel(writer, sheet_name=model_name, index=False)

        except PermissionError:
            #Printing error to warn user
            print(f"Error: Close the Excel file ({filename}) before running the script again.")

    #Printing a confirmation to ensure user results are logged
    print(f"Logged results for {model_name}: Train MSE={mse_train:.4f}, Test MSE={mse_test:.4f}, Unseen MSE={mse_unseen:.4f}")


In [ ]:
#Using the function to log results from RFR model
log_results(
    model_name="RFR_PCA",
    params=best_params,
    mse_train=mse_train,  
    r2_train=r2_train,
    mse_test=mse_test,
    r2_test=r2_test,
    mse_unseen=mse_unseen,
    r2_unseen=r2_unseen
)